In [1]:
# This file serves as a starting template for other scripts.
import glob
import os
import sys
import random
import time
import numpy as np
import cv2
import open3d as o3d    # Open3D is used for 3D visualization of point clouds
from matplotlib import cm   # Colormap for point cloud visualization

# Add the carla egg file to the python path
try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

client = carla.Client('localhost', 2000)
# client.reload_world()
world = client.get_world()

blueprint_library = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()
start_point = spawn_points[0]
random_start_point = random.choice(spawn_points)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# reset world
# client.reload_world()

In [3]:
# Create Actor List
# Function to clean up all actors
def cleanUp():
    for actor in actor_list:
        actor.destroy()
    print("All cleaned up!")

actor_list = []

In [4]:
# Spawn vehicle
vehicle_bp = blueprint_library.filter('vehicle.*')[0]
# vehicle = world.spawn_actor(vehicle_bp, random_start_point)
vehicle = world.spawn_actor(vehicle_bp, start_point)
actor_list.append(vehicle)

In [5]:
# # start autopilot
# vehicle.set_autopilot(True)

In [6]:
# Setup Sensors

# Camera Sensor
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_transform = carla.Transform(carla.Location(z=2.0))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)
actor_list.append(camera)

# gnss Sensor
gnss_bp = blueprint_library.find('sensor.other.gnss')
gnss = world.spawn_actor(gnss_bp, carla.Transform(), attach_to=vehicle)
actor_list.append(gnss)

# imu Sensor
imu_bp = blueprint_library.find('sensor.other.imu')
imu = world.spawn_actor(imu_bp, carla.Transform(), attach_to=vehicle)
actor_list.append(imu)

# collision Sensor, event based
collision_bp = blueprint_library.find('sensor.other.collision')
collision = world.spawn_actor(collision_bp, carla.Transform(), attach_to=vehicle)
actor_list.append(collision)

# lane invasion Sensor, event based
lane_invasion_bp = blueprint_library.find('sensor.other.lane_invasion')
lane_invasion = world.spawn_actor(lane_invasion_bp, carla.Transform(), attach_to=vehicle)
actor_list.append(lane_invasion)

# obstacle Sensor, event based
obstacle_bp = blueprint_library.find('sensor.other.obstacle')
obstacle_bp.set_attribute('distance', '10')
obstacle_bp.set_attribute('hit_radius', '0.5')
obstacle = world.spawn_actor(obstacle_bp, carla.Transform(), attach_to=vehicle)
actor_list.append(obstacle)

In [7]:
# Setup sensor callbacks
def rgb_callback(image, data_dict):
    data_dict['rgb_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

def gnss_callback(data, data_dict):
    data_dict['gnss'] = [data.latitude, data.longitude, data.altitude]

def imu_callback(data, data_dict):
    # accelerometer = radians, gyroscope = (mp/s)^2, compass = radians
    data_dict['imu'] = [data.accelerometer, data.gyroscope, data.compass]

def collision_callback(event, data_dict):
    data_dict['collision'] == True

def lane_invasion_callback(event, data_dict):
    data_dict['lane_invasion'] == True

def obstacle_callback(event, data_dict, camera, k_mat):
    if 'static' not in event.other_actor.type_id:
        data_dict['obstacle'].append({'transform': event.other_actor.type_id, 'frame': event.frame})

    world_2_camera = np.array(camera.get_transform().get_inverse_matrix())
    image_point = get_image_point(event.other_actor.get_transform().location, k_mat, world_2_camera)
    if 0 < image_point[0] < image_width and 0 < image_point[1] < image_height:
        cv2.circle(data_dict['rgb_image'], tuple(image_point), 10, (0, 0, 255), 3)

In [8]:
def build_projection_matrix(w, h, fov):
    focal = w / (2.0 * np.tan(fov * np.pi / 360.0))
    K = np.identity(3)
    K[0, 0] = K[1, 1] = focal
    K[0, 2] = w / 2.0
    K[1, 2] = h / 2.0
    return K

def get_image_point(location, k_mat, world_2_camera):
    # Calculate 2D projection of 3D coordinate

    # Format carla.Position object input coordinate
    point = np.array([location.x, location.y, location.z, 1])
    # transform to camera coordinate
    point_camera = np.dot(world_2_camera, point)

    # Change IE4 coordinate system to standard
    # (x, y, z) -> (y, -z, x)
    # and remove 4th component
    point_camera = [point_camera[1], -point_camera[2], point_camera[0]]

    # Project to 2D with camera matrix
    point_image = np.dot(k_mat, point_camera)
    # normalize
    point_image[0] /= point_image[2]
    point_image[1] /= point_image[2]

    return tuple(map(int, point_image[0:2]))

world_2_camera = np.array(camera.get_transform().get_inverse_matrix())

# Get attributes from camera
image_width = camera_bp.get_attribute('image_size_x').as_int()
image_height = camera_bp.get_attribute('image_size_y').as_int()
fov = camera_bp.get_attribute('fov').as_float()

# Build projection matrix
k_mat = build_projection_matrix(image_width, image_height, fov)

In [9]:
# Set up travel path for GNSS control testing
# Get map information
town_map = world.get_map()
print(town_map.name)

# Get waypoints
roads = town_map.get_topology()
print(roads)


Carla/Maps/Town10HD_Opt
[(<carla.libcarla.Waypoint object at 0x0000026CCA9CE0A0>, <carla.libcarla.Waypoint object at 0x0000026CCA9CD770>), (<carla.libcarla.Waypoint object at 0x0000026CCA9CD700>, <carla.libcarla.Waypoint object at 0x0000026CCA9CD690>), (<carla.libcarla.Waypoint object at 0x0000026CCA9CD5B0>, <carla.libcarla.Waypoint object at 0x0000026CCA9CD2A0>), (<carla.libcarla.Waypoint object at 0x0000026CCA9CD070>, <carla.libcarla.Waypoint object at 0x0000026CCA9CD150>), (<carla.libcarla.Waypoint object at 0x0000026CCA9CD0E0>, <carla.libcarla.Waypoint object at 0x0000026CCA9CCF90>), (<carla.libcarla.Waypoint object at 0x0000026CCA9CD000>, <carla.libcarla.Waypoint object at 0x0000026CCA9CCF20>), (<carla.libcarla.Waypoint object at 0x0000026CCA9CCDD0>, <carla.libcarla.Waypoint object at 0x0000026CCA9CCEB0>), (<carla.libcarla.Waypoint object at 0x0000026CCA9CCE40>, <carla.libcarla.Waypoint object at 0x0000026CCA9CCC80>), (<carla.libcarla.Waypoint object at 0x0000026CCA9CCCF0>, <carla

In [10]:
# get gnss data for map
gnss_data = []
for road in roads:
    for waypoint in road:
        gnss_data.append([waypoint.transform.location.x, waypoint.transform.location.y, waypoint.transform.location.z])
# print(gnss_data)
print((gnss_data))

[[109.92987823486328, -9.334196090698242, 0.0], [109.97942352294922, -22.602590560913086, 0.0], [106.42996215820312, -9.358065605163574, 0.0], [106.47980499267578, -22.654197692871094, 0.0], [102.98018646240234, -22.705795288085938, 0.0], [102.93003845214844, -9.381519317626953, 0.0], [99.4805679321289, -22.757402420043945, 0.0], [99.43012237548828, -9.405388832092285, 0.0], [19.330636978149414, 140.9599151611328, 0.0], [109.30042266845703, 82.96246337890625, 0.0], [19.35021209716797, 137.4599609375, 0.0], [105.80050659179688, 82.93859100341797, 0.0], [102.30058288574219, 82.91472625732422, 0.0], [19.36977767944336, 133.96002197265625, 0.0], [98.80066680908203, 82.89085388183594, 0.0], [19.389352798461914, 130.46006774902344, 0.0], [109.50276184082031, 53.29315185546875, 0.0], [109.57832336425781, 42.213409423828125, 0.0], [106.00284576416016, 53.269283294677734, 0.0], [106.07840728759766, 42.18954086303711, 0.0], [106.00284576416016, 53.269283294677734, 0.0], [106.07840728759766, 42.1

In [11]:
# # make a path for the vehicle to follow
# path = []
# for i in range(0, len(gnss_data), 10):
#     path.append(carla.Location(x=gnss_data[i][0], y=gnss_data[i][1], z=gnss_data[i][2]))
# print(path)


In [13]:
# Make a route from one GNSS position to another and use global route planner to navigate
# Get the start and end points
#import code for route planning, need networkx and numpy modules
sys.path.append('C:\CARLA_0.9.15\PythonAPI\carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner

# plan route and visually draw it in the simulator
sampling_resolution = 2
grp = GlobalRoutePlanner(town_map, sampling_resolution)

start_point = carla.Location(x=gnss_data[0][0], y=gnss_data[0][1], z=gnss_data[0][2])
end_point = carla.Location(x=gnss_data[-1][0], y=gnss_data[-1][1], z=gnss_data[-1][2])

route = grp.trace_route(start_point, end_point)

# Draw route in simulator
for i in range(len(route)):
    world.debug.draw_string(route[i][0].transform.location, 'O', draw_shadow=False,
                            color=carla.Color(r=255, g=0, b=0), life_time=120.0,
                            persistent_lines=True)


In [14]:
# display image in agnss sensorCV display window
image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

collision_counter = 20
lane_invasion_counter = 20

sensor_data = {'rgb_image': np.zeros((image_h, image_w, 4)),
               'collision' : False,
                'lane_invasion' : False,
                'obstacle' : [],
                'gnss' : [0, 0, 0],
                'imu' : {
                        'gyro': carla.Vector3D(),
                        'accel': carla.Vector3D(),
                        'compass': 0
                        }
                }
cv2.namedWindow('Camera RGB', cv2.WINDOW_AUTOSIZE)
cv2.imshow('Camera RGB', sensor_data['rgb_image'])
cv2.waitKey(1)

camera.listen(lambda image: rgb_callback(image, sensor_data))
gnss.listen(lambda data: gnss_callback(data, sensor_data))
imu.listen(lambda data: imu_callback(data, sensor_data))
collision.listen(lambda event: collision_callback(event, sensor_data))
lane_invasion.listen(lambda event: lane_invasion_callback(event, sensor_data))
obstacle.listen(lambda event: obstacle_callback(event, sensor_data, camera, k_mat))

font = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10, 50)
fontScale = 0.5
fontColor = (255, 255, 255)
lineType = 2
thickness = 2

def draw_compass(img, theta):
    compass_center = (700, 100)
    compass_size = 50

    cardinal_directions = [
        ('N', [0, -1]),
        ('E', [1, 0]),
        ('S', [0, 1]),
        ('W', [-1, 0])
    ]

    for car_dir in cardinal_directions:
        cv2.putText(sensor_data['rgb_image'], car_dir[0],
                    (int(compass_center[0] + 1.2 * compass_size * car_dir[1][0]),
                     int(compass_center[1] + 1.2 * compass_size * car_dir[1][1])),
                     font, fontScale, fontColor, lineType, thickness)
        
    compass_point = (int(compass_center[0] + compass_size * np.sin(theta)), int(compass_center[1] - compass_size * np.cos(theta)))
    cv2.line(img, compass_center, compass_point, (255, 255, 255), 3)

while True:
    cv2.putText(sensor_data['rgb_image'], 'Latitude: ' + str(sensor_data['gnss'][0]),
                (10,30), font, fontScale, fontColor, lineType, thickness)
    
    cv2.putText(sensor_data['rgb_image'], 'Longitude: ' + str(sensor_data['gnss'][1]),
                (10, 50), font, fontScale, fontColor, lineType, thickness)
    
    cv2.putText(sensor_data['rgb_image'], 'Altitude: ' + str(sensor_data['gnss'][2]),
                (10, 70), font, fontScale, fontColor, lineType, thickness)
    
    # accel = sensor_data['imu']['accel'] - carla.Vector3D(x=0, y=0, z=9.81)

    # cv2.putText(sensor_data['rgb_image'], 'Acceleration: ' + str(accel.length()),
    #             (10, 90), font, fontScale, fontColor, lineType, thickness)
    
    # cv2.putText(sensor_data['rgb_image'], 'Gyroscope: ' + str(sensor_data['imu']['gyro'].length()),
    #             (10, 120), font, fontScale, fontColor, lineType, thickness)
    
    # cv2.putText(sensor_data['rgb_image'], 'Compass: ' + str(sensor_data['imu']['compass']),
    #             (10, 140), font, fontScale, fontColor, lineType, thickness)
    
    # draw_compass(sensor_data['rgb_image'], sensor_data['imu']['compass'])

    if sensor_data['collision']:
        collision_counter -= 1
        if collision_counter <=1:
            sensor_data['collision'] = False
        cv2.putText(sensor_data['rgb_image'], 'Collision Detected!',
                    (250, 300), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3, 2)
    else:
        collision_counter = 20

    if sensor_data['lane_invasion']:
        lane_invasion_counter -= 1
        if lane_invasion_counter <= 1:
            sensor_data['lane_invasion'] = False
        cv2.putText(sensor_data['rgb_image'], 'Lane Invasion Detected!',
                    (250, 300), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3, 2)
        
    # render window
    cv2.imshow('Camera RGB', sensor_data['rgb_image'])
    if cv2.waitKey(1) == ord('q'):
        break

camera.stop()
gnss.stop()
imu.stop()
collision.stop()
lane_invasion.stop()
obstacle.stop()
cv2.destroyAllWindows()
# cleanUp()

# Test point starts here
